In [ ]:
import os
import sys

# 把项目根目录加入 Python 路径（必须）
project_root = os.path.abspath("..")
sys.path.append(project_root)

print("Project root:", project_root)

from utils.seed import set_seed
set_seed(42)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from models import MLP, SimpleCNN, ResNet18, ViTTiny
from trainer import Trainer
from data import get_emnist_dataloaders

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
num_classes = 10
train_loader, val_loader, num_classes = get_emnist_dataloaders(
    data_dir="../data",
    batch_size=64,
    split="balanced"
)

In [ ]:
print(f'训练例数量:{len(train_loader)}') # 存储了元组的“列表”
print(f'测试例数量:{len(val_loader )}')

In [ ]:
# 初始化模型
model = MLP(num_classes=num_classes).to(device)

# 优化器 & 损失函数
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# TensorBoard
writer = SummaryWriter("../logs/mlp_exp1")

# Trainer
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    writer=writer,
    num_classes=num_classes,
    model_name="mlp"
)

# 训练循环
epochs = 20

for epoch in range(epochs):
    
    train_loss, train_acc = trainer.train_one_epoch(train_loader, epoch)
    val_loss, val_auc = trainer.validate(val_loader, epoch)

    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {train_loss:.4f}, "
        f"Train Acc: {train_acc:.4f}, "
        f"Val Loss: {val_loss:.4f}, "
        f"Val AUC: {val_auc:.4f}"
    )

